In [1]:
# Install llmcompressor Tool
!pip install llmcompressor==0.5.2 -q


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
#Validate LLM Compressor is installed
!pip list | grep llmcompressor

llmcompressor                            0.5.2


In [3]:
# Run Python script to quantise the model

from quantiser import quantise_model

# You can view this code by opening the Python file from the File Explorer. 
quantise_model("TinyLlama/TinyLlama-1.1B-Chat-v1.0", scheme="W4A16")

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/app-root/lib64/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/opt/app-root/lib64/python3.11/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


2025-11-09T23:04:53.465794+0000 | reset | INFO - Compression lifecycle reset
2025-11-09T23:04:53.468437+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-11-09T23:04:53.500973+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-11-09T23:04:53.501839+0000 | IndependentPipeline | INFO - Inferred `DataFreePipeline` for `QuantizationModifier`


Calibrating weights: 100%|██████████| 293/293 [00:00<00:00, 285731.47it/s]

2025-11-09T23:04:53.511108+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-11-09T23:04:53.511625+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
2025-11-09T23:04:53.514632+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Quantised model saved to: TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16


'TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16'

In [6]:
# Review contents of the file 
!ls -lsah TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16

total 2.1G
4.0K drwxr-sr-x. 2 1000870000 1000870000 4.0K Nov  9 23:05 .
4.0K drwxr-sr-x. 3 1000870000 1000870000 4.0K Nov  9 23:04 ..
4.0K -rw-r--r--. 1 1000870000 1000870000  675 Nov  9 23:04 config.json
4.0K -rw-r--r--. 1 1000870000 1000870000  124 Nov  9 23:04 generation_config.json
2.1G -rw-r--r--. 1 1000870000 1000870000 2.1G Nov  9 23:05 model.safetensors
4.0K -rw-r--r--. 1 1000870000 1000870000  130 Nov  9 23:05 recipe.yaml


In [7]:
# Evaluate accuracy of the model. We will be using lm-eval script in vLLM.
!pip install vllm==0.9.1 lm-eval==0.4.7 -q


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Evaluate unquantized model
# !lm-eval --model vllm --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1 \
# --limit 250 --tasks hellaswag --num_fewshot 5 --batch_size 5
!time lm-eval \
  --model vllm \
  --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1,gpu_memory_utilization=0.8 \
  --limit 250 \
  --tasks hellaswag \
  --num_fewshot 5 \
  --batch_size 5

In [ ]:
# Now we run the same command against the post-quantized model.
# Note: You will need specify the Tokenizer explicitly.

!time lm-eval \
  --model vllm \
  --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16,tokenizer=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1,gpu_memory_utilization=0.8 \
  --limit 250 \
  --tasks hellaswag \
  --num_fewshot 5 \
  --batch_size 5

# !time lm-eval --model vllm --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0-W4A16,tokenizer=TinyLlama/TinyLlama-1.1B-Chat-v1.0,tensor_parallel_size=1 \
# --limit 250 --tasks hellaswag --num_fewshot 5 --batch_size 5

By comparsing the pre-quantized-eval and post-quantized-eval,

The value reduces from 0.572 to 0.564

Accuracy Recovery = Post-quantized-accuracy / Pre-quantized-accuracy.

So, the recovery here is 0.564 / 0.572 * 100% = 98.6%

Acutally, you have to run more than 500,000 evaluations on quantized models, which retains about 99% accuracy.